In [1]:
def lcs(strings):
    strings = sorted(strings, key=len)
    len0 = len(strings[0])
    if len0 == 0:
        return strings[0]

    for currentLength in range(len0, 0, -1):
        for startIndex in range(len0 - currentLength + 1):
            foundString = strings[0][startIndex : startIndex + currentLength]
            found = False
            string = strings[1]
            for startIndex2 in range(len(string) - currentLength + 1):
                if string[startIndex2:startIndex2+currentLength] == foundString:
                    found = True
                    break
            if not found:
                break
            if found:
                return foundString
    return str('')

def dif(strings):
    lcs_len = len(lcs(strings))
    return (strings[0][lcs_len:], strings[1][lcs_len:])

In [2]:
lcs(['aasdfbcd','bcd'])

'bcd'

In [3]:
lines = []
with open("task2_lemmas_train") as f:
    for line in f:
        lines.append(line.split(','))

In [4]:
data=[]
for line in lines[1:]:
    for j in range(2, len(line)):
        inf_part = line[j].split('+')
        data.append([line[0], line[1], inf_part[0], inf_part[1][0]])

endings = [(data[i][0], dif((data[i][1], data[i][2]))[0],
           dif((data[i][1], data[i][2]))[1] , data[i][3]) for i in range(len(data))]


In [6]:
import pandas as pd

data = pd.DataFrame(data)
endings = pd.DataFrame(endings)

In [7]:
data.head()

,0,1,2,3
0,1,vergognerete,vergognare,V
1,2,amnistiavate,amnistiare,V
2,3,menomazione,menomazione,N
3,4,sfaldavamo,sfaldare,V
4,5,sfodererei,sfoderare,V


In [8]:
endings.head()

,0,1,2,3
0,1,erete,are,V
1,2,vate,re,V
2,3,,,N
3,4,vamo,re,V
4,5,erei,are,V


Научимся определять по слову его часть речи

In [51]:
feature = data[1].tolist()
target = data[3].tolist()

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(lowercase=True, analyzer='char', ngram_range=(3, 6))
count_vectorizer.fit(feature)
feature_matrix = count_vectorizer.transform(feature)

In [54]:
from sklearn.linear_model import SGDClassifier
POS_classifier = SGDClassifier(loss='hinge', penalty='l2')

In [55]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(POS_classifier, feature_matrix, target, scoring =  'accuracy', cv = 3).mean()
print(score)

0.945623150756


In [56]:
POS_classifier.fit(feature_matrix, target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

Для каждой части речи научимся по слову определять его окончание

In [18]:
keys = ('A', 'N', 'V')

In [19]:
ending_lists = {key: sorted(list(set(endings[1][endings[3]==key])), key=len, reverse=True)
                for key in keys}

In [20]:
def identify_ending(word, key):
    for end in ending_lists[key]:
        if(word.endswith(end)):
            return end

Для каждой части речи по окончанию научимся определять окончание инфинитива

In [45]:
from collections import Counter
ending_pairs = {}
for key in keys:
    ending_pairs[key] = {
        end: Counter(endings[2][(endings[3]==key)&(endings[1]==end)].tolist()).most_common(1)[0][0]
        for end in ending_lists[key]
    }

In [44]:
for word in ending_lists['A']:
    print(word, Counter(endings[2][(endings[3]=='A') & (endings[1]==word)].tolist()).most_common(1)[0][0])

migliori buono
aggiori rande
aggiore rande
massimo grande
massime grande
massima grande
iedevo oprassedere
nente rre
venti re
centi rre
nenti rre
otevo ipercuotere
vente re
cente rre
ente ire
rici ore
enti ire
rice ore
nte re
nti re
he o
vo re
hi o
e o
o sere
a o
i o
 


In [46]:
ending_pairs

{'A': {'': '',
  'a': 'o',
  'aggiore': 'rande',
  'aggiori': 'rande',
  'cente': 'rre',
  'centi': 'rre',
  'e': 'o',
  'ente': 'ire',
  'enti': 'ire',
  'he': 'o',
  'hi': 'o',
  'i': 'o',
  'iedevo': 'oprassedere',
  'massima': 'grande',
  'massime': 'grande',
  'massimo': 'grande',
  'migliori': 'buono',
  'nente': 'rre',
  'nenti': 'rre',
  'nte': 're',
  'nti': 're',
  'o': 'sere',
  'otevo': 'ipercuotere',
  'rice': 'ore',
  'rici': 'ore',
  'vente': 're',
  'venti': 're',
  'vo': 're'},
 'N': {'': '',
  '-chiave': 'o-chiave',
  'a': 'o',
  'ces': 'x',
  'e': 'a',
  'e-estati': 'a-estate',
  'e-gol': 'a-gol',
  'e-lavoro': 'a-lavoro',
  'e-ordinanze': 'a-ordinanza',
  'e-paga': 'a-paga',
  'e-partito': 'a-partito',
  'edonne': 'adonna',
  'eforti': 'aforte',
  'ei': 'io',
  'en': 'an',
  'esorti': 'asorte',
  'he': 'a',
  'he-dati': 'a-dati',
  'heforti': 'aforte',
  'hi': 'o',
  'i': 'o',
  'i-chiave': 'o-chiave',
  'i-italia': 'a-italia',
  'i-leninismi': 'o-leninismo',
  'i-p

Всё готово для решения задачи!

In [69]:
test = pd.read_csv('task2_lemmas_test')

In [70]:
test_feature = test['X'].tolist()
test_feature_matrix = count_vectorizer.transform(feature)

In [71]:
preds = POS_classifier.predict(test_feature_matrix)

In [72]:
test['key'] = preds
test.head()

test_endings = [identify_ending(item[1], item[2]) for item in test.as_matrix()]

In [77]:
test['ending'] = test_endings
inf_endings = [ending_pairs[item[2]][item[3]] for item in test.as_matrix()]

In [78]:
test['inf_ending'] = inf_endings

In [82]:
words = test['X']
infs = [words[i][:-len(test_endings[i])] + inf_endings[i] for i in range(len(words))]

In [90]:
test['inf'] = infs
test['Category'] = test['inf'] + '+' + test['key']

In [91]:
test

,Id,X,key,ending,inf_ending,inf,Category
0,1,gettonan,V,an,e,gettone,gettone+V
1,2,incidentali,A,i,o,incidentalo,incidentalo+A
2,3,involtino,V,ino,are,involtare,involtare+V
3,4,lievi,N,i,o,lievo,lievo+N
4,5,comunistizzasse,V,sse,re,comunistizzare,comunistizzare+V
5,6,vidimerebbe,V,erebbe,are,vidimare,vidimare+V
6,7,imbrodan,V,an,e,imbrode,imbrode+V
7,8,strillar,V,,e,e,e+V
8,9,cifrasti,V,sti,re,cifrare,cifrare+V
9,10,compassavano,V,vano,re,compassare,compassare+V


In [87]:
answer = pd.read_csv('task2_lemmas_sample_submission')

In [92]:
answer['Category'] = test['Category']

In [93]:
answer

,Id,Category
0,1,gettone+V
1,2,incidentalo+A
2,3,involtare+V
3,4,lievo+N
4,5,comunistizzare+V
5,6,vidimare+V
6,7,imbrode+V
7,8,e+V
8,9,cifrare+V
9,10,compassare+V


In [94]:
answer.to_csv("lemmas_answer.csv", sep=',', index=False)